# 5. Metaprogramowanie

# 5.1 Dekoratory

In [2]:
def generate_adder(a):
    def add(b):
        return a + b
    return add


add_10 = generate_adder(10)
add_5 = generate_adder(5)
print(add_10(100))
print(add_5(100))

110
105


In [6]:
def shouter(func):  # func == hellome
    def wrapper():  # hellome_decorated == wrapper
        print('before', func.__name__)
        func()
        print('after', func.__name__)
    return wrapper

def hellome():
    print("hellome")
    
hellome_decorated = shouter(hellome)

In [7]:
hellome()

hellome


In [8]:
hellome_decorated()

before hellome
hellome
after hellome


In [9]:
def shouter(func):  # func == hellome
    def wrapper():  # hellome_decorated == wrapper
        print('before', func.__name__)
        result = func()
        print('after', func.__name__)
        return result
    return wrapper

def hellome():
    print("hellome")
    return 42
    
hellome_decorated = shouter(hellome)

result = hellome_decorated()
print('result', result)

before hellome
hellome
after hellome
result 42


In [15]:
def shouter(func):  # func == hellome
    def wrapper():  # hellome_decorated == wrapper
        print('before', func.__name__)
        result = func()
        print('after', func.__name__)
        return result
    return wrapper

@shouter
def hellome():
    print("hellome")
    return 42

# hellome = shouter(hellome)

result = hellome()
print('result', result)

before hellome
hellome
after hellome
result 42


**Ćwiczenie.**
Zmień implementację dekoratora `@shouter` tak, aby działał także z funkcjami przyjmującymi argumenty.

In [24]:
def shouter(func):  # func == hellome
    def wrapper(*args, **kwargs):  # hellome == wrapper
        print('before', func.__name__)
        result = func(*args, **kwargs)
        print('after', func.__name__)
        return result
    return wrapper

@shouter
def hellome(a, b):
    print("hellome")
    return a+b

result = hellome(20, b=22)
print('result', result)  # ==> 'result', 42

before hellome
hellome
after hellome
result 42


In [18]:
%doctest_mode

Exception reporting mode: Plain
Doctest mode is: ON


In [33]:
from functools import wraps

def shouter(func):  # func == hellome
    @wraps(func)
    def wrapper(*args, **kwargs):  # hellome == wrapper
        print('before', func.__name__)
        result = func(*args, **kwargs)
        print('after', func.__name__)
        return result
    return wrapper

@shouter
def hellome(a, b):
    '''jreha ajfkekwaf.'''
    print("hellome")
    return a+b

result = hellome(20, b=22)
print('result', result)  # ==> 'result', 42
print(hellome.__doc__)

before hellome
hellome
after hellome
result 42
jreha ajfkekwaf.


In [38]:
from functools import wraps

class shouter:
    def __init__(self, func):
        self.func = func
        
    def __call__(self, *args, **kwargs):
        print('before', self.func.__name__)
        result = self.func(*args, **kwargs)
        print('after', self.func.__name__)
        return result

@shouter
def hellome(a, b):
    '''jreha ajfkekwaf.'''
    print("hellome")
    return a+b

result = hellome(20, b=22)
print('result', result)  # ==> 'result', 42

before hellome
hellome
after hellome
result 42


**Ćwiczenie.**
Napisz dekorator `@cached`:
1. Używając zagnieżdżonych funkcji
2. Używając klas

In [44]:
import functools

def cached(f):
    cache = {}
    @functools.wraps(f)
    def wrapper(*args):
        try:
            return cache[args]
        except KeyError:
            result = f(*args)
            cache[args] = result
            return result
    return wrapper

@cached
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-2) + fib(n-1)
        
%timeit fib(30)

392 ns ± 0.541 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


# 5.1.4 Dekoratory parametryzowane

In [47]:
def parametrized_shouter(msg):
    def shouter(func):
        @functools.wraps(func)
        def wrapper():
            print('Before', msg)
            result = func()
            print('After', msg)
            return result
        return wrapper
    return shouter

@parametrized_shouter('blah')
def hellome():
    print('hellome')
    
hellome()

Before blah
hellome
After blah


**Ćwiczenie.**
Napisz dekorator @head(n), który powoduje, że zwracane jest tylko n pierwszych kolekcji, którą zwraca dekorowana funkcja.

In [57]:
def head(limit):
    def decor(func):
        @functools.wraps(func)
        def wrapper(*args):
            print('start head')
            result = func(*args)[:limit]
            print('end head')
            return result
        return wrapper
    return decor

@head(6)
def upper(tekst):
    return tekst.upper()

print(upper('qwertyuiop'))  # ==> 'QWERTY'

start head
end head
QWERTY


In [51]:
def limit(limit):
    def decor(func):
        @functools.wraps(func)
        def wrapper(*args):
            for no, i in enumerate(func(*args)):
                if no == limit:
                    break
                yield i
        return wrapper
    return decor


@limit(10)
def count(start):
    while True:
        yield start
        start += 1
        
cc = count(5)
for _, c in zip(range(15), cc):
    print(c)

5
6
7
8
9
10
11
12
13
14


### Wiele dekoratorów

In [58]:
@shouter
@head(6)
def upper(text):
    print('upper')
    return text.upper()

print(upper('qwertyuiop'))

before upper
start head
upper
end head
after upper
QWERTY


### Dekoratory klas

In [60]:
def addID(original_class):
    orig_init = original_class.__init__
    counter = 0
    def __init__(self, *args, **kwargs):
        nonlocal counter
        print("addID init")
        self._id = counter
        counter += 1
        orig_init(self, *args, **kwargs)
    original_class.__init__ = __init__
    return original_class

@addID
class Person:
    def __init__(self, name):
        print('Person class init')
        self._name = name
        
p = Person('Jan')

addID init
Person class init


In [61]:
print(p._name)
print(p._id)

Jan
0


In [62]:
p2 = Person('John')
print(p2._id)

addID init
Person class init
1
